# Загрузка данных анкет и проведение МАИ, анализ полученных результатов
Данные: заполненные анкеты экспертов (таблицы Excel) 

In [21]:
import re
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
from random import randint

In [22]:
df = pd.read_excel(r'C:\Users\user\предпосылки\Экс11.xlsx')
df.head()

,Тип нарушения,Документация,Экспертиза промышленной безопасности,Готовность к ликвидации последствий аварий и инцидентов,Оформление информационными знаками и знаками безопасности,Содержание объектов и оборудования,Охранные зоны (ОЗ),Организация рабочего процесса и безопасности персонала
0,Документация,1.000000,3.000000,3.0,1.000000,3.0,4.0,3
1,Экспертиза промышленной безопасности,0.333333,1.000000,5.0,7.000000,3.0,5.0,3
2,Готовность к ликвидации последствий аварий и и...,0.333333,0.200000,1.0,5.000000,1.0,5.0,3
3,Оформление информационными знаками и знаками б...,1.000000,0.142857,0.2,1.000000,3.0,1.0,3
4,Содержание объектов и оборудования,0.333333,0.333333,1.0,0.333333,1.0,7.0,3


In [23]:
print('Число строк и столбцов в наборе данных:\n', df.shape)

Число строк и столбцов в наборе данных:
 (7, 8)


In [24]:
# типы столбцов 
df.dtypes

Тип нарушения                                                 object
Документация                                                 float64
Экспертиза промышленной безопасности                         float64
Готовность к ликвидации последствий аварий и инцидентов      float64
Оформление информационными знаками и знаками безопасности    float64
Содержание объектов и оборудования                           float64
Охранные зоны (ОЗ)                                           float64
Организация рабочего процесса и безопасности персонала         int64
dtype: object

In [25]:
#удаление столбца типа object для дальнейшей работы с фреймом 
df.pop('Тип нарушения') 
print(df)

   Документация   Экспертиза промышленной безопасности  \
0       1.000000                              3.000000   
1       0.333333                              1.000000   
2       0.333333                              0.200000   
3       1.000000                              0.142857   
4       0.333333                              0.333333   
5       0.250000                              0.200000   
6       0.333333                              0.333333   

   Готовность к ликвидации последствий аварий и инцидентов  \
0                                           3.000000         
1                                           5.000000         
2                                           1.000000         
3                                           0.200000         
4                                           1.000000         
5                                           0.200000         
6                                           0.333333         

   Оформление информационными знаками 

In [26]:
# типы столбцов 
df.dtypes

Документация                                                 float64
Экспертиза промышленной безопасности                         float64
Готовность к ликвидации последствий аварий и инцидентов      float64
Оформление информационными знаками и знаками безопасности    float64
Содержание объектов и оборудования                           float64
Охранные зоны (ОЗ)                                           float64
Организация рабочего процесса и безопасности персонала         int64
dtype: object

In [27]:
# также меням тип данных Документации
df['Документация '] = df['Документация '].astype(float)

In [28]:
df.loc[:, df.columns[df.dtypes == 'float64']].head(7)

,Документация,Экспертиза промышленной безопасности,Готовность к ликвидации последствий аварий и инцидентов,Оформление информационными знаками и знаками безопасности,Содержание объектов и оборудования,Охранные зоны (ОЗ)
0,1.000000,3.000000,3.000000,1.000000,3.000000,4.000000
1,0.333333,1.000000,5.000000,7.000000,3.000000,5.000000
2,0.333333,0.200000,1.000000,5.000000,1.000000,5.000000
3,1.000000,0.142857,0.200000,1.000000,3.000000,1.000000
4,0.333333,0.333333,1.000000,0.333333,1.000000,7.000000
5,0.250000,0.200000,0.200000,1.000000,0.142857,1.000000
6,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333


In [29]:
DF_1=df[:].copy()

In [30]:
#Найдем сумму всех столбцов, для того чтобы в дальнейшем пересчитать полученные значения
df.sum ()

Документация                                                  3.583333
Экспертиза промышленной безопасности                          5.209524
Готовность к ликвидации последствий аварий и инцидентов      10.733333
Оформление информационными знаками и знаками безопасности    15.666667
Содержание объектов и оборудования                           11.476190
Охранные зоны (ОЗ)                                           23.333333
Организация рабочего процесса и безопасности персонала       19.000000
dtype: float64

In [31]:
# Считаем значения для нашей матрицы, путем деления каждого значения в столбце на сумму столбца
df['new_Документация'] = df['Документация '].astype(float).div(df['Документация '].sum())
df['new_Экспертиза'] = df['Экспертиза промышленной безопасности'].astype(float).div(df['Экспертиза промышленной безопасности'].sum())
df['new_Готовность'] = df['Готовность к ликвидации последствий аварий и инцидентов'].astype(float).div(df['Готовность к ликвидации последствий аварий и инцидентов'].sum())
df['new_Оформление'] = df['Оформление информационными знаками и знаками безопасности'].astype(float).div(df['Оформление информационными знаками и знаками безопасности'].sum())
df['new_Содержание'] = df['Содержание объектов и оборудования '].astype(float).div(df['Содержание объектов и оборудования '].sum())
df['new_ОЗ'] = df['Охранные зоны (ОЗ)'].astype(float).div(df['Охранные зоны (ОЗ)'].sum())
df['new_Организация'] = df['Организация рабочего процесса и безопасности персонала'].astype(float).div(df['Организация рабочего процесса и безопасности персонала'].sum())

In [32]:
df.head()

,Документация,Экспертиза промышленной безопасности,Готовность к ликвидации последствий аварий и инцидентов,Оформление информационными знаками и знаками безопасности,Содержание объектов и оборудования,Охранные зоны (ОЗ),Организация рабочего процесса и безопасности персонала,new_Документация,new_Экспертиза,new_Готовность,new_Оформление,new_Содержание,new_ОЗ,new_Организация
0,1.000000,3.000000,3.0,1.000000,3.0,4.0,3,0.279070,0.575868,0.279503,0.063830,0.261411,0.171429,0.157895
1,0.333333,1.000000,5.0,7.000000,3.0,5.0,3,0.093023,0.191956,0.465839,0.446809,0.261411,0.214286,0.157895
2,0.333333,0.200000,1.0,5.000000,1.0,5.0,3,0.093023,0.038391,0.093168,0.319149,0.087137,0.214286,0.157895
3,1.000000,0.142857,0.2,1.000000,3.0,1.0,3,0.279070,0.027422,0.018634,0.063830,0.261411,0.042857,0.157895
4,0.333333,0.333333,1.0,0.333333,1.0,7.0,3,0.093023,0.063985,0.093168,0.021277,0.087137,0.300000,0.157895


In [33]:
# Получили новую таблицу теперь отделим значения нашей матрицы от основной таблицы
df_new = df[['new_Документация', 'new_Экспертиза', 'new_Готовность', 'new_Оформление', 'new_Содержание', 'new_ОЗ', 'new_Организация']]
df_new

,new_Документация,new_Экспертиза,new_Готовность,new_Оформление,new_Содержание,new_ОЗ,new_Организация
0,0.279070,0.575868,0.279503,0.063830,0.261411,0.171429,0.157895
1,0.093023,0.191956,0.465839,0.446809,0.261411,0.214286,0.157895
2,0.093023,0.038391,0.093168,0.319149,0.087137,0.214286,0.157895
3,0.279070,0.027422,0.018634,0.063830,0.261411,0.042857,0.157895
4,0.093023,0.063985,0.093168,0.021277,0.087137,0.300000,0.157895
5,0.069767,0.038391,0.018634,0.063830,0.012448,0.042857,0.157895
6,0.093023,0.063985,0.031056,0.021277,0.029046,0.014286,0.052632


In [34]:
#Вычислим средние значение по строке и запишем полученный результот в mean
df_new['mean'] = df_new.mean(axis=1)
df_new.head()

C:\Users\user\anaconda3\envs\envpy3.5\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,new_Документация,new_Экспертиза,new_Готовность,new_Оформление,new_Содержание,new_ОЗ,new_Организация,mean
0,0.279070,0.575868,0.279503,0.063830,0.261411,0.171429,0.157895,0.255572
1,0.093023,0.191956,0.465839,0.446809,0.261411,0.214286,0.157895,0.261603
2,0.093023,0.038391,0.093168,0.319149,0.087137,0.214286,0.157895,0.143293
3,0.279070,0.027422,0.018634,0.063830,0.261411,0.042857,0.157895,0.121588
4,0.093023,0.063985,0.093168,0.021277,0.087137,0.300000,0.157895,0.116641


In [35]:
#проверка, среднее значение в сумме должно быть равно 1
df_new['mean'].sum()

1.0

In [36]:
# Далее проведем проверку на согласованность, 
#для этого создадим из нашей полученной выше таблицы матрицы для дальнейшего их перемножения
df_new8 =df_new['mean']

# удаляем из нашей таблицы значениея столбцы mean
#df_new.pop('mean')

df_new1 =DF_1.loc[0]

df_new2 =DF_1.loc[1]

df_new3 =DF_1.loc[2]

df_new4 =DF_1.loc[3]

df_new5 =DF_1.loc[4]

df_new6 =DF_1.loc[5]

df_new7 =DF_1.loc[6]

In [37]:
#Проверка на согласованность
res1 = np.dot(df_new1,df_new8)
print(res1)
res2 = np.dot(df_new2,df_new8)
print(res2)
res3 = np.dot(df_new3,df_new8)
print(res3)
res4 = np.dot(df_new4,df_new8)
print(res4)
res5 = np.dot(df_new5,df_new8)
print(res5)
res6 = np.dot(df_new6,df_new8)
print(res6)
res7 = np.dot(df_new7,df_new8)
print(res7)

2.303367944754551
2.683585378951385
1.4246748888136223
0.9816461994558812
1.0075208964258011
0.4716567756749756
0.36240991045537385


In [38]:
sum_new = res1 + res2 + res3 + res4 + res5 + res6 + res7
print(sum_new)

9.234861994531592


In [39]:
# Вычислем значения CI, RI, CR
CI = (sum_new - 7)/(7-1)
print(CI)
RI = (1.98*(7-2))/7
print(RI)
CR = CI/RI
print(CR)

0.3724769990885986
1.4142857142857144
0.2633675751131505


In [40]:
# Получилось, что значение CR=0.26, что большее 0,1 значит матрицу нельзя считать согласованной
# Проделаем аналогичные действия с другими экспертами